In [1]:
import pandas as pd
import json
import fornax
import matplotlib
import networkx as nx

%matplotlib inline

# Performing a Fuzzy Graph Query using Fornax

Im this tutorial we will perform a fuzzy search on the dataset curated during the first tutorial.

You need to run tutorial 1 before doing this tutorial otherwise you will not be able to load any of the data produced.

## Step 1: Load the data from the first tutorial

In [2]:
nodes_df = pd.read_csv('./nodes.csv')
edges_df = pd.read_csv('./edges.csv')

In [3]:
nodes_df.head()

,label,type,uid
0,Selene,0,87770955
1,Doctor Doom,0,2073821878
2,Viper,0,396175249
3,Rhino,0,279892555
4,Sin,0,2062678112


In [4]:
edges_df.head()

,end,start
0,839851079,87770955
1,685373387,2073821878
2,1988120854,396175249
3,608208951,396175249
4,603235485,279892555


## Step 2: Create a target graph

We create a target graph by creating a graph handle and loading the nodes and edges into the graph.

We can attach metadata to the nodes using keyword arguments of `GraphHandle.add_nodes`.
`id_src` is a special field which specifies a unique id for each node. By default `id_src` is a continuous range of integers starting at zero.

In [5]:
conn = fornax.Connection()
conn.open()
target_graph = fornax.GraphHandle.create(conn)
target_graph.add_nodes(id_src=nodes_df['uid'], label=nodes_df['label'], type_=nodes_df['type'])
target_graph.add_edges(edges_df['start'], edges_df['end'])

## Step 3: Create a query graph

Creating a query graph is exactly the same as creating a target graph. Because this query graph is small we don't load any values from file.

In [6]:
query_graph = fornax.GraphHandle.create(conn)
query_graph.add_nodes(id_src=[0, 1, 2], label=['hulk', 'lady', 'storm'])
query_graph.add_edges([0, 1], [1, 2])

## Step 4: Create a query

A query needs a set of correspondances between the query and target graph.
They can be anything provided that they are scored between 0 and 1.
Below we create a correspondance between a query node and a target node. If the query node label is a substring of a target node label the correspondance scores 1, otherwise it is zero we don't specify it explicitly.

Just like graph edges, matches are specified using the `src_id` of source and target nodes respectivly.

In [7]:
matches = []
for i in nodes_df[nodes_df['label'].str.contains("(?i)hulk")]['uid']:
    matches.append((0, i, 1))
for i in nodes_df[nodes_df['label'].str.contains('(?i)lady')]['uid']:
    matches.append((1, i, 1))
for i in nodes_df[nodes_df['label'].str.contains('(?i)storm')]['uid']:
    matches.append((2, i, 1))
sources, targets, weights = zip(*matches)

In [8]:
matches[:10]

[(0, 2087196931, 1),
 (0, 2142361735, 1),
 (0, 2020667501, 1),
 (0, 959673558, 1),
 (0, 970313367, 1),
 (0, 329519748, 1),
 (0, 1056123890, 1),
 (0, 560425637, 1),
 (1, 1639515098, 1),
 (1, 995920086, 1)]

A query needs a two graph handles.
Once created we can insert the matches.

## Exacute the query

The query produces a python dictionary that can be serialised to json.
Each `graph` in the result uses the [node link fromat](https://networkx.github.io/documentation/stable/reference/readwrite/generated/networkx.readwrite.json_graph.node_link_data.html#networkx.readwrite.json_graph.node_link_data) for graphs.

The resulting graphs contain
* The query nodes
* The optimal target nodes matching the query nodes
* Edges between the query nodes
* Edges between the relavent target nodes
* The matches between the query graph and optimum target nodes

In [12]:

query = fornax.QueryHandle.create(conn, query_graph, target_graph)
query.add_matches(sources, targets, weights)
    results = query.execute(n=1)

IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed [SQL: 'INSERT INTO "query" (query_id, start_graph_id, end_graph_id) VALUES (?, ?, ?)'] [parameters: (0, 0, 0)] (Background on this error at: http://sqlalche.me/e/gkpj)

In [ ]:
print(json.dumps(results, indent=4))

# Draw the result using networkx

In [ ]:
graph = results['graphs'][0]
G = nx.json_graph.node_link_graph(graph)
nodes = G.nodes()
cols = []
for k in G.nodes():
    if nodes[k]['type'] == 'query':
        cols.append('r')
    elif nodes[k]['type'] == 'target':
        cols.append('b')

In [ ]:
nx.draw(
    G, 
    with_labels=True, 
    labels={n['id']:n['label'] for n in graph['nodes']}, 
    node_color=cols
)

## Write the resulting supgraph match using another graph format 

In [ ]:
nx.write_gexf(G, 'my_graph.gexf')